In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py exists and contains:
# get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

<h1 align="center">Decorators and descriptors</h1>

In [ ]:
class count_calls:
    def __init__(self, f):
        self.count = 0
        self.f = f
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f'Count nb {self.count} to {self.f}')
        return self.f(*args, **kwargs)

# Equivalent to:
# add_up = count_calls(add_up)
@count_calls
def add_up(x, y, *, a, b):
    return x + y + a + b

add_up(1, 2, a = 2, b = 3)
add_up(4, 5, a = 6, b = 7)
add_up(8, 9, a = 10, b = 11)

In [ ]:
def count_calls(f):
    count = 0
    def wrap(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Count nb {count} to {f}')
        return f(*args, **kwargs)
    return wrap

# Equivalent to:
# add_up = count_calls(add_up)
@count_calls
def add_up(x, y, *, a, b):
    return x + y + a + b

add_up(1, 2, a = 2, b = 3)
add_up(4, 5, a = 6, b = 7)
add_up(8, 9, a = 10, b = 11)

In [ ]:
def count_calls_starting_from(start = 0):
    def count_calls(f):
        count = start
        def wrap(*args, **kwargs):
            nonlocal count
            count += 1
            print(f'Count nb {count} to {f}')
            return f(*args, **kwargs)
        return wrap
    return count_calls

# Equivalent to:
# add_up = count_calls_starting_from(1)(add_up)
@count_calls_starting_from(1)
def add_up(x, y, *, a, b):
    return x + y + a + b

add_up(1, 2, a = 2, b = 3)
add_up(4, 5, a = 6, b = 7)
add_up(8, 9, a = 10, b = 11)

In [ ]:
def count_calls(cls):
    def wrap(datum):
        wrap.count += 1
        print(f'Count nb {wrap.count} to {cls}')
        return cls(datum)
    wrap.count = 0
    return wrap

# Equivalent to:
# C = count_calls(C)
@count_calls
class C:
    def __init__(self, datum):
        self.datum = datum

I1, I2, I3 = C(11), C(12), C(13)
I1.datum, I2.datum, I3.datum

In [ ]:
class C:
    count_1 = 0
    count_2 = 0  
    def __init__(self):
        C.count_1 += 1
        C.count_2 += 1 
    def display_count_1(mark):
        print('count_1' + mark, C.count_1)
    # Equivalent to:
    # display_count_2 = staticmethod(display_count_2)
    @staticmethod
    def display_count_2(mark):
        print('count_2' + mark, C.count_2)

I1, I2, I3 = C(), C(), C()
C.display_count_1(':')
C.display_count_2('...')
I2.display_count_2(' ')

In [ ]:
class C:
    count = 0 
    def __init__(self):
        C.count += 1 
    # Equivalent to:
    # display_count = classmethod(display_count)
    @classmethod
    def display_count(cls, mark):
        print(f'count for {cls.__name__}' + mark, C.count)

I1, I2, I3 = C(), C(), C()
C.display_count('...')
I2.display_count(':')

A __descriptor__ is any class with at least one of the three methods:
* \_\_get\_\_(self, instance, owner)
* \_\_set\_\_(self, instance, value)
* \_\_delete\_\_(self, instance)

It is called:
* a __data descriptor__ if it implements \_\_set\_\_()
* a __non-data descriptor__ if it does not implement \_\_set\_\_().

In [ ]:
class D:
    def __init__(self):
        self.datum = 'Descriptor datum'
    def __get__(self, instance, owner):
        print(self.datum)
        print(owner._datum)
        return instance._datum     
    def __set__(self, instance, value):
        self.datum = 'New descriptor datum'
        instance._datum = value   
    def __delete__(self, instance):
        print('Deleting instance datum')
        del instance._datum

class C:
    _datum = 'Owner datum'
    def __init__(self):
        self._datum = 'Instance datum'
    datum = D()

I = C()
I.datum
I.datum = 'New instance value'
I.datum
del I.datum
print()
I = C()
I.datum

In [ ]:
class DataDescriptorWithGet:
    def __get__(self, instance, owner):
        return 'X1'
    def __set__(self, instance, value):
        pass

class DataDescriptorWithoutGet:
    def __set__(self, instance, value):
        pass

class NonDataDescriptor:
    def __get__(self, instance, owner):
        return 'X3'

class C:
    x1 = DataDescriptorWithGet()
    x2 = DataDescriptorWithoutGet()
    x3 = NonDataDescriptor()
    def __init__(self):
        self.x1 = 'x1'
        self.x2 = 'x2'

I = C(); I.__dict__, I.x1, I.x2, I.x3
I.x1 = 'xx1'; I.__dict__, I.x1
I.__dict__['x1'] = 'xx1'; I.__dict__, I.x1
I.x2 = 'xx2'; I.__dict__, I.x2
I.__dict__['x2'] = 'xx2'; I.__dict__, I.x2
I.x3 = 'x3'; I.__dict__, I.x3
I.__dict__['x3'] = 'xx3'; I.__dict__, I.x3

In [ ]:
class C:
    def __init__(self, datum):
        self._datum = datum
    # Equivalent to:
    # datum = property(fget = datum, fset = None, fdel = None, doc = None)
    # Using that form would set C.datum.__doc__ to the value of doc;
    # with the decorator, that value is instead 'For illustration purposes'.
    @property
    def datum(self):
        'For illustration purposes'
        print('You asked for the value of datum')
        return self._datum
    # C.datum is now a descriptor, with in particular
    # - the built-in methods getter, setter and deleter;
    # - the functions fget, fset, fdel;
    # - the method-wrappers __get__, __set__, __delete__. 
    # C.datum.__get__ is a method wrapper of C.datum.fget (the function above).
    #
    # Equivalent to:
    # datum = datum.setter(datum)
    # Returns a copy of datum with C.datum.fset assigned the function below.
    # C.datum.__set__ is a method wrapper of C.datum.fset.
    @datum.setter
    def datum(self, value):
        print('You want to modify the value of datum')
        self._datum = value
    # Equivalent to:
    # datum = datum.deleter(datum)
    # Returns a copy of datum with C.datum.fdel assigned the function below.
    # C.datum.__delete__ is a method wrapper of C.datum.fdel.
    @datum.deleter
    def datum(self):
        print('You have decided to delete datum')
        del self._datum

I = C(3)
I.datum
I.datum = 4
print()
I.datum
del I.datum

In [ ]:
class D1:
    def __init__(self, x):
        self.x = x
    def __get__(self, instance, cls):
        return self.x
    def __set__(self, instance, value):
        self.x = value

class D2:
    def __init__(self, x):
        self.x = x  
    def __get__(self, instance, cls):
        return self.x

class C:
    d11 = D1('d11 in descriptor')
    d12 = D1('d12 in descriptor')
    d13 = D1('d13 in descriptor')
    d21 = D2('d21 in descriptor')
    d22 = D2('d22 in descriptor')
    d23 = D2('d23 in descriptor')
    d31 = 'd31 in class'
    d32 = 'd32 in class'
    d33 = 'd33 in class'
    def __str__(self):
        return 'In class'
    def __getattribute__(self, attribute):
        return f'{attribute}!'
    def __getattr__(self, attribute):
        return f'{attribute}...'
    def __init__(self):
        self.__str__ = lambda self: 'In instance'
        try:
            self.__dict__['whatever'] = None
        except TypeError as error:
            print('Here is what happens:', error)
        self.d13 = 'd13 not assigned to instance'
        self.d23 = 'd23 not assigned to instance'
        self.d33 = 'd33 not assigned to instance'
    
c = C(); print(c)
c.__dict__
c.d1
c.d21
c.d22
c.d31
c.d32
c.whatever

In [ ]:
class D1:
    def __init__(self, x):
        self.x = x
    def __get__(self, instance, cls):
        return self.x
    def __set__(self, instance, value):
        self.x = value

class D2:
    def __init__(self, x):
        self.x = x  
    def __get__(self, instance, cls):
        return self.x

class C:
    d11 = D1('d11 in descriptor')
    d12 = D1('d12 in descriptor')
    d13 = D1('d13 in descriptor')
    d21 = D2('d21 in descriptor')
    d22 = D2('d22 in descriptor')
    d23 = D2('d23 in descriptor')
    d31 = 'd31 in class'
    d32 = 'd32 in class'
    d33 = 'd33 in class'
    def __str__(self):
        return 'In class'
    def __getattr__(self, attribute):
        return f'{attribute}...'
    def __init__(self):
        self.__str__ = lambda self: 'In instance'
        self.__dict__['d12'] = "d12 not assigned to instance's __dict__"
        self.__dict__['d22'] = "d22 directly assigned to instance's __dict__"
        self.__dict__['d32'] = "d32 directly assigned to instance's __dict__"
        self.d13 = 'd13 in descriptor via set'
        self.d23 = "d23 undirectly assigned to instance's __dict__"
        self.d33 = "d33 undirectly assigned to instance's __dict__"

c = C(); print(c)
c.__dict__
c.d11
c.d12
c.d13
c.d21
c.d22
c.d23
c.d31
c.d32
c.d33
c.whatever